# Поиск генов под отбором

**1. Подготовка исходных данных.**  
В качестве исходных данных нам нужно три организма, для которых есть геномные сборки (.fasta/.fna), аннотация (.gff) и белки (.fasta/.faa)

Предположим, что они у нас называются genome1.fna, annot1.gff и protein1.faa для первого организма и т.д. по аналогии.

**2. Кластеризация белков с целью обнаружения однокопийных ортологов.**  
Установка программы:  
conda install proteinortho  

Запуск кластеризации:  
proteinortho6.pl protein1.faa protein2.faa protein3.faa

Эта команда выдаст файл my-project.proteinortho.tsv, содержащий все ортологичные группы. Чтобы вытащить только интересующие нас однокопийные ортологи (т.е. гены, в каждом организме присутвующие только в одном экземпляре), удобно воспользоваться awk:


**3. Подготовка данных для программы PAML**  
По айдишкам из файла single_copy.tsv необходимо вытащить соответствующие этим айдишникам белковые и кодирующие нуклеотидные последовательности (CDS).

На самом деле эта самая геморройная задача этого проекта, но я надеюсь, что вы справитесь :)

In [2]:
from collections import defaultdict
import re

def get_revcomp(sequence):
    '''Return reverse complementary sequence.
    >>> complementary('AT CG')
    'CGAT'
    '''
    c = dict(zip('ATCGNatcgn~[]', 'TAGCNtagcn~]['))
    return ''.join(c.get(nucleotide, '') for nucleotide in reversed(sequence))

def read_fasta_file(fasta_file):              
    fasta = {}                                
    header = None                            
    with open(fasta_file) as fh:              
        for i, line in enumerate(fh):         
            line = line.strip()               
            if line.startswith(">"):          
                if header:                   
                    fasta[header] = "".join(seq)
                header = line[1:].split()[0]         
                seq = []                      
            else:
                seq.append(line)              
        if header:                            
            fasta[header] = "".join(seq)      
    return fasta


def gtf2seq(gtf_file, fasta_dict, is_ncbi=False):
    cds_seq = defaultdict(list)
    with open(gtf_file) as fh:
        for line in fh:
            if line.startswith("#"):
                continue
            line = line.strip().split("\t")

            if line[2] != "CDS":
                continue
            
            scaf_id = line[0]
            if is_ncbi:
                gene_name = re.findall("Name=(\S+?);", line[-1])
                if not gene_name:
                    print(line)
                    continue
                gene_name = gene_name[0]
            else:
                gene_name = line[-1].split()[1].replace('"', "").replace(';', "") 
            strand = line[6]
            if strand == "+":
                cds_start = int(line[3])-1
                cds_end = int(line[4])
            else:
                
            
            key = (gene_name, strand)
            
            if cds_start > cds_end:
                cds_start, cds_end = cds_end, cds_start ### order them!
            exon = fasta_dict[scaf_id][cds_start:cds_end] 
            value = (cds_start, exon)
            cds_seq[key].append(value)
            
    genes = {}
    for key in cds_seq:
        (gene_name, strand) = key
        exons = cds_seq[key]
        exons.sort() ### ну вдруг у тебя там координаты сломались в gtf/gff, оно бывает, а так мы уверены что они по порядку будут
        gene = "".join([x[1] for x in exons]) # тут мы просто берем сиквенсы и скипаем служебные старты для сортировки
        if strand == "-":
            gene = get_revcomp(gene)
        ### параноидально чекаем что у нас gene_name уникальное, если тут упадет, то у тебя паралоги собираются по имени в одну корзинку
        assert gene_name not in genes
        genes[gene_name] = gene
    return genes

In [4]:
# пропишите тут все пути к соответствующим файлам

sngl_copy = "/path/to/file"

genome1_file = "/path/to/file"
annot1_file = "/path/to/file"
protein1_file = "/path/to/file"

genome2_file = "/path/to/file"
annot2_file = "/path/to/file"
protein2_file = "/path/to/file"

genome3_file = "/path/to/file"
annot3_file = "/path/to/file"
protein3_file = "/path/to/file"

In [5]:
# cчитываем из файлов разные типы данных и 
# сохраняем их в виде словарей

genome1 = read_fasta_file(genome1_file)
genome2 = read_fasta_file(genome2_file)
genome3 = read_fasta_file(genome3_file)

protein1 = read_fasta_file(protein1_file)
protein2 = read_fasta_file(protein2_file)
protein3 = read_fasta_file(protein3_file)

# если геном не в формате с NCBI, то не забудьте убрать аргумент is_ncbi

cds1 = gff2seq(annot1_file, genome1, is_ncbi=False)
cds2 = gff2seq(annot2_file, genome2, is_ncbi=False)
cds3 = gff2seq(annot3_file, genome3, is_ncbi=False)

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/file'

In [ ]:
## дальше довольно кривой код, feel free to привести его в порядок

## каждый кластер однокопийных ортологов мы записываем в отдельный файл

## обязательно нужно знать порядок столбцов в файле single_copy.txt

# их нужно сохранить в отдельную папку cds_clusters (не забудьте ее создать!)

counter = 0
with open(sngl_copy) as orthologs:
    for line in orthologs:
        file = "/path/to/output/files/cds_clusters/cluster_%s.dna.mfa" % str(counter)
        counter += 1
        with open(file, "w") as fh:
            if line.startswith("#"):
                continue
            line = line.strip().split()

#             print(len(cds2[line[3]]))
#             print(len(cds2[line[4]]))
#             print(len(cds3[line[5]]))

            fh.write(">organism1_" + line[3] + "\n")
            fh.write(cds1[line[3]] + "\n")
            fh.write(">organism2_" + line[4] + "\n")
            fh.write(cds2[line[4]] + "\n")
            fh.write(">organism3_" + line[5] + "\n")
            fh.write(cds3[line[5]] + "\n")

In [ ]:
# то же самое проделываем для белковых кластеров

# их нужно сохранить в отдельную папку prot_clusters (не забудьте ее создать!)

counter = 0
with open(sngl_copy) as orthologs:
    for line in orthologs:
        file = "/path/to/output/files/prot_clusters/cluster_%s.prot.mfa" % str(counter)
        counter += 1
        with open(file, "w") as fh:
            if line.startswith("#"):
                continue
            line = line.strip().split()

#             print(len(protein1[line[3]]))
#             print(len(protein2[line[4]]))
#             print(len(protein3[line[5]]))

            fh.write(">organism1_" + line[3] + "\n")
            fh.write(cds1[line[3]] + "\n")
            fh.write(">organism2_" + line[4] + "\n")
            fh.write(cds2[line[4]] + "\n")
            fh.write(">organism3_" + line[5] + "\n")
            fh.write(cds3[line[5]] + "\n")

Теперь у нас есть папки с распарсенными кодирующими последовательностями и белками. Следующий шаг - выравнивание.

**4. Выравнивание белковой последовательности.**

Чтобы получить выравнивание по кодонам, сначала выравнивают белки между собой в одном кластере. Почему именно белки, а не CDS, - они более консервативные, последовательности гораздо легче выравнять правильно


Запустила команды с clustal omega для того, чтобы выровнять каждый кластер белка.  
clustalo -i {input_file} -o {output_file}

Для распараллеливания большого количества:  
mkdir /path/to/output/files/prot_clusters/aligned/

ls /path/to/output/files/prot_clusters/ | grep mfa | xargs -I {} -P 120 sh -c "clustalo -i /path/to/output/files/prot_clusters/{} -o /path/to/output/files/prot_clusters/aligned/{}.aligned"


**3. Выравнивание по кодонам**

Далее используем pal2nal, он переводит белковое выравнивание в выравнивание по кодонами + убирает все гэпы (потому что они создают false positive для PAML) + переводит выравнивание в фoрмат, который хочет PAML)  
pal2nal.pl cluster_1.prot.mfa.aligned cluster_1.dna.mfa -output paml -nogap > cluster_1.pal2nal  

Но так как у нас есть несколько сотен файлов, нужно распараллеливать. Для этого нам нужен файл с записью всех комманд:


In [7]:
counter = 0
with open(sngl_copy) as orthologs:
    with open("tasks.txt", "w") as fh:
        for line in orthologs:
            file = "cluster_%s" % str(counter)
            counter += 1

            if line.startswith("#"):
                continue
                
            command = f"pal2nal.pl prot_clusters/aligned/{file}.prot.mfa.aligned cds_clusters/{file}.dna.mfa -output paml -nogap > pal2nal/{file}.pal2nal"
            fh.write(command + "\n")

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/file'

Для распараллеливания большого количества:  
less tasks.txt | xargs -I {} -P 120 sh -c "{}"

Когда все упомянутое сделано, можно запускать PAML

Мануал: DOI: 10.1007/978-1-4939-1438-8_4 

Если успеетее сделать все вышеописанное за сегодняшний день, я скину инструкцию по PAML.